In [13]:
import os
import sys
import pandas as pd
import pymfe
import glob
from pymfe.mfe import MFE
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [2]:
cwd = os.getcwd()
cwd_back = os.path.dirname(cwd)
data_path = os.path.join(cwd_back, 'tcc_')

In [26]:
#Parametros que virão da API:

nvezes = 1 #nvezes = numero de repetições do experimento para cada um dos conjuntos de dados
nreduce = [0.1, 0.12, 0.15] #nreduce = lista com as porcentagens de redução para cada conjunto de dados


In [28]:
data_list = os.path.join(data_path, 'data') #data = Pasta local onde se encontra os dataset, arquivos .csv 

all_files = glob.glob(data_list + "/*.csv") #fazendo o carregamento dos datasets da pasta 'data' com os arquivos .csv

mfe = MFE(groups="all", summary=["mean", "sd", "min", "median", "max"]) # MFE Groups, passando 'ALL' com parametro para todos os grupos de medidas
mfe_sumary = MFE(summary=["min", "median", "max"]) # MFE Summary, passando parametros de sumarização 'minimo', 'media' e 'maximo'
mfe_quantiles = MFE(summary=["quantiles"])  # MFE Summary, passando parametros de sumarização "quantiles"

files = [] # vetor para armazenar os Dataframes com os dados de cada dataset carregados
i = 0 # variavel contadora para contabilizar o numero de repetições do experimento
for file_ in all_files: #laço de repetição para ler todos os arquivos da pasta local 'data'
    filepath_ = os.path.join(data_path, 'data', file_) # extraindo e armazenando o caminho e nome do arquivo .csv
    files.append(pd.read_csv(file_,encoding = "ISO-8859-1", decimal=",", header=0)) #transformando o csv e um DataFrame pandas e armazenando-o em vetor
    print(filepath_) #printando o caminho e os nomes dos arquivos carregados

nfiles = 0    #variavel contadora para armazenar o numero de arquivos carregados
for df in files: #laço para iterara o vetor com os datafremes que foram armazenados
    nfiles = nfiles+1 #contando os arquivos
    for i in range(nvezes): #laço para repetir o experimento n vezes
        
    ########################## inicio processamento dos datasets originais no MFE ######################################
        print("\nArquivo numero: ",nfiles) #printando o numero do arquivo 
        print('Número de instâncias: {}\nNúmero de atributos: {}\n'.format(len(df), len(df.columns))) #printando informação sobre o daframe numero de instancias e atributos
        list_df_origin = df.values.tolist() # transformando o dataframe em uma lista para passar para o MFE
        
        mfe.fit(list_df_origin) # passando a lista de dados para o MFE opção Group (All)
        ft = mfe.extract() # extraindo as metricas do MFE Group (All)
        print('\nMFE - Groups (ALL) - DF Original ',"\n") #mostrando o parametro Groups= All passado para o MFE
        df_ft = pd.DataFrame (ft) #transformando a saida do MFE em um dataframe
        print(df_ft,"\n") #mostrando o DataFrame de saida do MFE opção Groups All
       
        
        mfe_sumary.fit(list_df_origin) # passando a lista de dados para o MFE opção sumary (min, med, max)
        ft_sumary = mfe_sumary.extract() # extraindo as metricas do MFE sumary (min, med, max)
        print('MFE - summary=["mean", "sd", "min", "median", "max"] - DF Original',"\n") #mostrando o parametro sumary (min, med, max) MFE
        df_ft_sumary = pd.DataFrame (ft_sumary) #transformando a saida do MFE em um dataframe
        print(df_ft_sumary,"\n") #mostrando o DataFrame de saida do MFE opção sumary (min, med, max)

        mfe_quantiles.fit(list_df_origin) # passando a lista de dados para o MFE opção sumary (quantiles)
        ft_quantiles = mfe_quantiles.extract() # extraindo as metricas do MFE sumary (quantiles)
        print('MFE - summary = quantiles - DF Original',"\n") #mostrando o parametro sumary (quantiles)
        df_ft_quantiles = pd.DataFrame (ft_quantiles) #transformando a saida do MFE em um dataframe
        print(df_ft_quantiles,"\n")  #mostrando o DataFrame de saida do MFE opção sumary (quantiles)

        
        for reduce in nreduce: #laço para realizar a redução de dados a partir dos datasets originais nreduce =  valor em porcento das reduções
            
        ########################## inicio processamento dos datasets Reduzidos no MFE ######################################
            print("\n tamanho da Redução: ", reduce)# mostrando o tamanho da redução
            random_df = df.sample(frac=reduce) #realizando a redução aleatoria do dataset
            list_df_reduce = random_df.values.tolist() # transformando o dataframe em lista para enviar para o MFE
          
            
            mfe.fit(list_df_reduce) # passando a lista de dados para o MFE opção Group (All)
            ft = mfe.extract() # extraindo as metricas do MFE Group (All)
            print('\nMFE - Groups (ALL) - DF Reduzido ',"\n") #mostrando o parametro Groups= All passado para o MFE
            df_ft_reduce = pd.DataFrame (ft) #transformando a saida do MFE em um dataframe
            print(df_ft_reduce,"\n") #mostrando o DataFrame de saida do MFE opção Groups All
            
            mfe_sumary.fit(list_df_reduce) # passando a lista de dados para o MFE opção sumary (min, med, max)
            ft_sumary = mfe_sumary.extract() # extraindo as metricas do MFE sumary (min, med, max)
            print('MFE - summary=["min", "median", "max"] - DF Reduzido',"\n")  #mostrando o parametro sumary (min, med, max) MFE
            df_ft_sumary_reduce = pd.DataFrame (ft_sumary) #transformando a saida do MFE em um dataframe
            print(df_ft_sumary_reduce,"\n")  #mostrando o DataFrame de saida do MFE opção sumary (min, med, max)

            mfe_quantiles.fit(list_df_reduce) # passando a lista de dados para o MFE opção sumary (quantiles)
            ft_quantiles = mfe_quantiles.extract() # extraindo as metricas do MFE sumary (quantiles)
            print('MFE - summary = quantiles - DF Reduzido',"\n") #mostrando o parametro sumary (quantiles)
            df_ft_quantiles_reduce = pd.DataFrame (ft_quantiles) #transformando a saida do MFE em um dataframe
            print(df_ft_quantiles_reduce,"\n")  #mostrando o DataFrame de saida do MFE opção sumary (quantiles)
            
        print("**********************Numero de Repetições da Redução: ",i+1,"***********************************************************\n")


C:\Users\danie\Documents\tcc_\data\Iris.csv
C:\Users\danie\Documents\tcc_\data\Iris1.csv

Arquivo numero:  1
Número de instâncias: 150
Número de atributos: 6


MFE - Groups (ALL) - DF Original  

             0               1                 2              3    \
0  attr_conc.max  attr_conc.mean  attr_conc.median  attr_conc.min   
1       0.905999        0.291637          0.239205       0.029905   

            4             5              6                7             8    \
0  attr_conc.sd  attr_ent.max  attr_ent.mean  attr_ent.median  attr_ent.min   
1      0.231374      5.033829       3.640018         4.038686      1.584963   

           9    ...      125        126         127       128         129  \
0  attr_ent.sd  ...  var.max   var.mean  var.median   var.min      var.sd   
1     1.387067  ...   1887.5  15.379563     0.02613  0.006667  170.187053   

           130           131             132          133         134  
0  wg_dist.max  wg_dist.mean  wg_dist.median  wg_dist.

MFE - summary = quantiles - DF Reduzido 

                     0                      1                      2    \
0  attr_conc.quantiles.0  attr_conc.quantiles.1  attr_conc.quantiles.2   
1                   0.08               0.256199               0.760418   

                     3                      4                     5    \
0  attr_conc.quantiles.3  attr_conc.quantiles.4  attr_ent.quantiles.0   
1                0.87345                    1.0                   1.0   

                    6                     7                     8    \
0  attr_ent.quantiles.1  attr_ent.quantiles.2  attr_ent.quantiles.3   
1               1.95439              3.228972              3.444027   

                    9    ...                 102                 103  \
0  attr_ent.quantiles.4  ...  t_mean.quantiles.0  t_mean.quantiles.1   
1              3.836592  ...                 0.0                 0.0   

                  104                 105                 106  \
0  t_mean.quantiles


MFE - Groups (ALL) - DF Reduzido  

             0               1                 2              3    \
0  attr_conc.max  attr_conc.mean  attr_conc.median  attr_conc.min   
1            1.0        0.615817          0.701425       0.077381   

            4             5              6                7             8    \
0  attr_conc.sd  attr_ent.max  attr_ent.mean  attr_ent.median  attr_ent.min   
1      0.329228      3.640224        2.71216         3.281728      0.996792   

           9    ...      125        126         127       128         129  \
0  attr_ent.sd  ...  var.max   var.mean  var.median   var.min      var.sd   
1     1.132756  ...   1714.6  37.361905    0.066667  0.066667  252.790719   

           130           131             132          133         134  
0  wg_dist.max  wg_dist.mean  wg_dist.median  wg_dist.min  wg_dist.sd  
1     2.969326      2.835368        2.857525     2.597668    0.116287  

[2 rows x 135 columns] 

MFE - summary=["min", "median", "max"] - DF

MFE - summary = quantiles - DF Reduzido 

                     0                      1                      2    \
0  attr_conc.quantiles.0  attr_conc.quantiles.1  attr_conc.quantiles.2   
1               0.039648               0.242884                0.60357   

                     3                      4                     5    \
0  attr_conc.quantiles.3  attr_conc.quantiles.4  attr_ent.quantiles.0   
1               0.787339                    1.0                   1.0   

                    6                     7                     8    \
0  attr_ent.quantiles.1  attr_ent.quantiles.2  attr_ent.quantiles.3   
1              1.952324              3.407962              3.808475   

                    9    ...                 102                 103  \
0  attr_ent.quantiles.4  ...  t_mean.quantiles.0  t_mean.quantiles.1   
1              4.095795  ...                 0.0                 0.0   

                  104                 105                 106  \
0  t_mean.quantiles